## 패키지 설치 및 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install selenium --upgrade
!pip install requests
from selenium import webdriver
from selenium.webdriver.common.by import By

import sys
!sudo add-apt-repository ppa:saiarcot895/chromium-beta
!sudo apt remove chromium-browser
!sudo snap remove chromium
!sudo apt install chromium-browser

!pip install tqdm
import pandas as pd
from tqdm import tqdm
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_experimental_option("detach", True)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

webdriver_service = Service('/usr/lib/chromium-browser/chromedriver')
driver = webdriver.Chrome(service=webdriver_service,options = options)

from time import sleep

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import requests
import re
import random
import requests
import json
from datetime import datetime, timedelta
import os
import csv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
PPA publishes dbgsym, you may need to include 'main/debug' component
Repository: 'deb https://ppa.launchpadcontent.net/saiarcot895/chromium-beta/ubuntu/ jammy main'
Description:
This PPA contains the latest Chromium Beta builds, with hardware video decoding enabled (hidden behind a flag), and support for Widevine (needed for viewing many DRM-protected videos) enabled.

== Hardware Video Decoding ==

To enable hardware video decoding, start Chromium with the --enable-features=VaapiVideoDecoder argument. To make this persistent, create a file at /etc/chromium-browser/customizations/92-vaapi-hardware-decoding with the following contents:

CHROMIUM_FLAGS="${CHROMIUM_FLAGS} --enable-features=VaapiVideoDecoder"

See also https://wiki.archlinux.org/title/

## 크롤링 함수

In [ ]:
# url 크롤링
def keyword_crawling(company, start_date, end_date, pbar=None):
    urls = set()
    s_date = start_date.replace('.', '')
    e_date = end_date.replace('.', '')
    year = start_date.split('.')[0]
    search_url = f"https://search.naver.com/search.naver?where=news&query={company}&sm=tab_opt&sort=0&photo=0&field=0&pd={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{s_date}to{e_date}&is_sug_officeid=0&office_category=0&service_area=0"
    driver.get(search_url)
    driver.implicitly_wait(10)
    before_h = driver.execute_script("return window.scrollY")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(random.uniform(2,3))
        # 스크롤 후 높이
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.info')))
        after_h = driver.execute_script("return window.scrollY")
        time.sleep(random.uniform(2,3))
        if after_h == before_h:
            naver_urls = driver.find_elements(By.CSS_SELECTOR, 'a.info')
            break
        before_h = after_h

    for naver_url in naver_urls:
        url = naver_url.get_attribute('href')
        match = re.search(r'\d{3}$', url)
        # 중복된 URL이 아니면 추가하고 파일에도 저장
        if "news.naver.com" in url and "sports" not in url and match and match.group() not in {"106", "107"} and match.group() in {"100", "101", "102", "103", "104", "105", "110"} and url not in urls:
            urls.add(url)
            if pbar is not None:
                pbar.update(1)

    with open(f'/content/drive/MyDrive/Kwargs/뉴스 크롤링/urls/{company}{start_date}.txt', 'w', encoding='utf-8') as f:
        for url in urls:
            f.write(url + '\n')
    print(f'{year}년 {company} url 저장 완료\n')
    return list(urls)

In [ ]:
# 기사 크롤링(카테고리, 내용, 제목 리턴)
def article_crawling(url):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"}
    original_html = requests.get(url, headers = headers)
    soup = BeautifulSoup(original_html.text, "html.parser")
    driver.implicitly_wait(10)

    # 카테고리
    category_mapping = {
    '100': '정치',
    '101': '경제',
    '102': '사회',
    '103': '생활문화',
    '104': '세계',
    '105': 'IT과학',
    '110': '오피니언'
    }
    match = re.search(r'sid=(\d+)', url)
    category = category_mapping.get(match.group(1))


    # 제목
    title = soup.select_one('#title_area')
    if title:
        title = title.text
    else:
        title = ''

    # 날짜
    try:
        datestamp = soup.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div:nth-child(1) > span")
        date = datestamp.attrs['data-date-time']
    except AttributeError:
        date = soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div:nth-child(1) > span')
        date = re.sub(r'<[^>]*>','',str(date))

    try:
        for tag in soup.find_all("span", {"class": "end_photo_org"}):
            tag.decompose()
        for tag in soup.find_all("strong", {"class": "media_end_summary"}):
            tag.decompose()
        for a_tag in soup.find_all('a'):
            a_tag.decompose()
    except AttributeError:
        pass


    # 내용
    content = soup.select_one("#newsct_article")
    if content:
        content = content.text.replace("\n","").replace("\t","")
    else:
        content = ''

    return content, title, date, category

In [ ]:
def clean_content(text):
    text = re.sub(r'Copyrights.*', '',text)
    text = re.sub(r'^\n+', '', text)
    text = re.sub(r"^\n", "", text)
    text = re.sub(r'\n+', '', text) #줄바꿈 태그 제거
    text = re.sub(r'\t+', '', text) # 탭 태그 제거
    text = re.sub('<[^>]*>','', text)
    text = re.sub(r'^\[|\]$', '',text) #앞뒤 [또는]제거
    text = re.sub('\s{2,}',' ',text) # 공백 2자리=>1자리로
    text = re.sub('[=+#/^$*※&ㆍ!』\\|\[\]\<\>`…》ⓒ☞■□ㅁ◆◇▶◀▷◁△▽▲▼○●━]', '', text)
    text = text.replace("""[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}""",'')
    text = text.replace('\"','')
    text = text.replace('무단복제 및 재배포 금지','')
    text = text.replace('무단전재 및 재배포 금지','')
    text = re.sub('\\n+','',text)
    text = text.replace('gt;','')
    text = text.replace('lt;','')
    text = text.replace('amp;','')
    return text

def clean_title(title):
    title = re.sub('<[^>]*>','', title)
    title = re.sub(r'^\[|\]$', '',title)
    title = title.replace('\"','')
    title = re.sub(r'\n+', '', title)
    return title

In [ ]:
# 2개월 단위 날짜 리스트 생성
def create_date_ranges(start_date, end_date):
    date_ranges = []
    current_date = start_date

    while current_date <= end_date:
        next_month = current_date.month + 2
        next_year = current_date.year + (next_month - 1) // 12
        next_month = (next_month - 1) % 12 + 1
        next_date = current_date.replace(year=next_year, month=next_month)

        if next_date > end_date:
            next_date = end_date
        date_ranges.append((current_date.strftime("%Y.%m.%d"), next_date.strftime("%Y.%m.%d")))
        current_date = next_date + timedelta(days=1)

    return date_ranges

start_date = datetime(2019, 1, 1)
end_date = datetime(2023, 12, 31)

date_ranges = create_date_ranges(start_date, end_date)

In [ ]:
# 체크포인트 로드하기
def load_checkpoint(checkpoint_path):
    try:
        with open(checkpoint_path, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            data = list(reader)
        return data
    except FileNotFoundError:
        return []

In [ ]:
def main():
    # 각자 회사이름 작성
    checkpoint_path = f'/content/drive/MyDrive/Kwargs/뉴스 크롤링/csv/{company}_news_data.csv'
    # 기존에 저장된 데이터 로드
    try:
        data = pd.read_csv(checkpoint_path)
        for start_date, end_date in date_ranges:
            file_name = f'{company}{start_date}.txt'
            file_path = f'/content/drive/MyDrive/Kwargs/뉴스 크롤링/urls/{file_name}'
            if os.path.exists(file_path):
                continue

            # 날짜 범위에 해당하는 데이터를 따로 저장할 데이터프레임 생성
            new_data = pd.DataFrame(columns=['title', 'date', 'content', 'category'])
            with tqdm(desc=f"Crawling urls {company} : {start_date} ~ {end_date}") as pbar:
                urls = keyword_crawling(company, start_date, end_date, pbar)
            for url in tqdm(urls, desc=f"Crawling news for {company} : {start_date} ~ {end_date}"):
                # 제목, 내용 크롤링
                content, title, date, category = article_crawling(url)
                title = clean_title(title)
                content = clean_content(content)

                if content and title and "포토" not in title:
                    new_data = pd.concat([new_data, pd.DataFrame({'title': [title], 'date': [date], 'content': [content],'category': [category]})], ignore_index=True)
                elif not content and not title:
                    print('추가해야될 url:', url, title, content)
                    with open(f'/content/drive/MyDrive/Kwargs/뉴스 크롤링/urls/추가url_{company}.txt', 'a', encoding='utf-8') as f:
                        f.write(url + '\n')

            new_data.to_csv(checkpoint_path, index=False, mode='a', header=not os.path.exists(checkpoint_path), encoding='utf-8')
            # 새로운 데이터를 기존 데이터프레임에 추가
            data = pd.concat([data, new_data], ignore_index=True)

        # 모든 데이터를 CSV 파일로 저장
        data.to_csv(checkpoint_path, index=False, mode='w', encoding='utf-8')

    except FileNotFoundError:
        data = pd.DataFrame(columns=['title', 'date', 'content', 'category'])
        for start_date, end_date in date_ranges:
        # 날짜 범위에 해당하는 데이터를 따로 저장할 데이터프레임 생성
            new_data = pd.DataFrame(columns=['title', 'date', 'content','category'])

            with tqdm(desc=f"Crawling urls {company} : {start_date} ~ {end_date}") as pbar:
                urls = keyword_crawling(company, start_date, end_date, pbar)
            for url in tqdm(urls, desc=f"Crawling news for {company} - {start_date.split('.')[0]}"):
                # 제목, 내용, 날짜, 카테고리 크롤링
                content, title, date, category = article_crawling(url)
                title = clean_title(title)
                content = clean_content(content)

                if content and title and "포토" not in title:
                    new_data = pd.concat([new_data, pd.DataFrame({'title': [title], 'date': [date], 'content': [content],'category': [category]})], ignore_index=True)
                elif not content and not title:
                    print('추가해야될 url:', url, title, content)
                    with open(f'/content/drive/MyDrive/Kwargs/뉴스 크롤링/urls/추가url_{company}.txt', 'a', encoding='utf-8') as f:
                        f.write(url + '\n')

            new_data.to_csv(checkpoint_path, index=False, mode='a', header=not os.path.exists(checkpoint_path), encoding='utf-8')
            # 새로운 데이터를 기존 데이터프레임에 추가
            data = pd.concat([data, new_data], ignore_index=True)
        # 모든 데이터를 CSV 파일로 저장
        data.to_csv(checkpoint_path, index=False, mode='w', encoding='utf-8')

## 실행

In [ ]:
company = '삼성SDI'#input("종목명:")
if __name__ == "__main__":
    main()

Crawling urls 삼성SDI : 2019.01.01 ~ 2019.03.01: 734it [24:52,  2.03s/it]


2019년 삼성SDI url 저장 완료



Crawling news for 삼성SDI - 2019:  75%|███████▌  | 553/734 [06:35<02:03,  1.46it/s]

## 기존 크롤링 결과 수정

In [ ]:
# 기존에 존재하는 url 텍스트파일에서 카테고리에 대해 삭제 함수 (크롤링 전부 완료한 회사만 이 함수 실행 시키기)
def edit_url(company):
    for start_date, end_date in date_ranges:
        with open(f'/content/drive/MyDrive/Kwargs/뉴스 크롤링/{company}{start_date}.txt', 'r', encoding='utf-8') as f:
            urls = f.read().splitlines()

        # 수정된 파일 경로를 정의하여 파일을 쓰기 모드로 엽니다.
        with open(f'/content/drive/MyDrive/Kwargs/뉴스 크롤링/{company}{start_date}.txt', 'w', encoding='utf-8') as f_out:
            for url in urls:
                match = re.search(r'\d{3}$', url)
                if match and match.group() in {"100", "101", "102", "103", "104", "105", "110"} and match.group() not in {"106", "107"}:
                    f_out.write(url + '\n')


# 기존에 크롤링 완료한 파일 중에서 다시 크롤링 해야되느 경우 실행
def edit_crawling(company):
    # 저장 경로 지정
    data_path = f'/content/drive/MyDrive/Kwargs/뉴스 크롤링/{company}_news_data.csv'
    data = pd.DataFrame(columns=['title', 'date', 'content', 'category'])

    for start_date, end_date in date_ranges:
        # 날짜 범위에 해당하는 데이터를 따로 저장할 데이터프레임 생성
        new_data = pd.DataFrame(columns=['title', 'date', 'content', 'category'])

        with open(f'/content/drive/MyDrive/Kwargs/뉴스 크롤링/{company}{start_date}.txt', 'r', encoding='utf-8') as f:
            urls = f.read().splitlines()
            for url in tqdm(urls, desc=f"Crawling news for {company} : {start_date} ~ {end_date}"):
                url = url.strip()
                # 제목, 내용, 날짜, 카테고리 크롤링
                content, title, date, category = article_crawling(url)
                title = clean_title(title)
                content = clean_content(content)
                if content and title and "포토" not in title:
                    new_data = pd.concat([new_data, pd.DataFrame({'title': [title], 'date': [date], 'content': [content],'category': [category]})], ignore_index=True)
                elif not content and not title:
                    print('추가해야될 url:', url, title, content)
                    with open(f'/content/drive/MyDrive/Kwargs/뉴스 크롤링/추가url_{company}.txt', 'a', encoding='utf-8') as f:
                        f.write(url + '\n')

        new_data.to_csv(data_path, index=False, mode='a', header=not os.path.exists(data_path), encoding='utf-8')
        # 새로운 데이터를 기존 데이터프레임에 추가
        data = pd.concat([data, new_data], ignore_index=True)
    # 모든 데이터를 CSV 파일로 저장
    data.to_csv(data_path, index=False, mode='w', encoding='utf-8')

In [ ]:
def main():
    # edit_url(company)
    edit_crawling(company)

In [ ]:
company = ''#input("종목명:")
if __name__ == "__main__":
    main()

Crawling urls 대한항공 : 2019.01.01 ~ 2019.03.01: 872it [19:48,  1.36s/it]


2019년 대한항공 url 저장 완료



Crawling news for 대한항공 - 2019:  18%|█▊        | 159/872 [02:11<13:29,  1.14s/it]

추가해야될 url: https://n.news.naver.com/mnews/article/015/0004086614?sid=101  


Crawling news for 대한항공 - 2019:  46%|████▌     | 399/872 [05:10<06:51,  1.15it/s]

...

Crawling news for 대한항공 - 2021: 100%|██████████| 931/931 [11:47<00:00,  1.32it/s]
Crawling urls 대한항공 : 2022.01.19 ~ 2022.03.19: 928it [19:30,  1.26s/it]


2022년 대한항공 url 저장 완료



Crawling news for 대한항공 - 2022:  10%|█         | 96/928 [01:13<12:42,  1.09it/s]

추가해야될 url: https://n.news.naver.com/mnews/article/421/0005936466?sid=101  


Crawling news for 대한항공 - 2022: 100%|██████████| 928/928 [11:26<00:00,  1.35it/s]
Crawling urls 대한항공 : 2022.03.20 ~ 2022.05.20: 0it [00:00, ?it/s]